In [112]:
import pandas as pd
import numpy as np
from sdv.single_table import GaussianCopulaSynthesizer
from sdv.metadata import Metadata

### Importing data

In [113]:
data = pd.read_csv('clean_data.csv')
data

,timestamp,role,frequency_of_travel,primary_purpose,travel_days,travel_hours,ISB_Service_trip_1,bus_stop_board_trip_1,bus_stop_alight_trip_1,day_of_the_week_trip_1,time_start_trip_1,travel_duration_trip_1,frequency_trip_1,punctuality_trip_1,cleanliness_trip_1,safety_trip_1,coverage_trip_1,crowdedness_trip_1,ISB_Service_trip_2,bus_stop_board_trip_2,bus_stop_alight_trip_2,day_of_the_week_trip_2,time_start_trip_2,travel_duration_trip_2,frequency_trip_2,punctuality_trip_2,cleanliness_trip_2,safety_trip_2,coverage_trip_2,crowdedness_trip_2,ISB_Service_trip_3,bus_stop_board_trip_3,bus_stop_alight_trip_3,day_of_the_week_trip_3,time_start_trip_3,travel_duration_trip_3,frequency_trip_3,punctuality_trip_3,cleanliness_trip_3,safety_trip_3,coverage_trip_3,crowdedness_trip_3,usage_influence_convenience,usage_influence_cost,usage_influence_lack_of_options,usage_influence_availability_of_parking,usage_influence_environmental,prioritize_frequency,prioritize_punctuality,prioritize_cleanliness,prioritize_safety,prioritize_bus_route_coverage,prioritize_crowdedness,top_3_frustrations,not_able_to_get_on,additional_features_frequency,additional_features_seats,additional_features_cleanliness,additional_features_comfortable,additional_features_route_coverage,additional_features_updates,issues_with_quality_of_info,special_events,seasonal_changes,seasonal_changes_specific,further_comments
0,10/1/2024 22:17:11,Undergraduate student,Daily,Commute to classes,"Monday, Tuesday, Wednesday, Thursday, Friday","0900 - 1000, 1000 - 1100, 1100 - 1200, 1200 - ...",A2,Opp NUSS,Ventus,Monday,09:30:00,15 - 20 minutes,Neutral,Neutral,Very Satisfied,Satisfied,Satisfied,3,D2,KR MRT,COM3,Friday,10:30:00,5 - 10 minutes,Neutral,Neutral,Very Satisfied,Satisfied,Satisfied,4.0,D2,COM3,LT27,Tuesday,14:30:00,10 - 15 minutes,Dissatisfied,Dissatisfied,Very Satisfied,Very Satisfied,Satisfied,3.0,1st,2nd,3rd,4th,5th,3rd,2nd,6th,5th,4th,1st,"Frequency of bus services, Capacity of the bus...",Occasionally,1st,4th,6th,5th,2nd,3rd,Yes,3,"No, service is consistent",NaN,More frequent buses during exam periods
1,10/2/2024 1:03:02,Undergraduate student,3 - 4 days a week,Commute to classes,"Monday, Wednesday, Thursday","0900 - 1000, 1100 - 1200, 1300 - 1400, 1500 - ...",D2,KR MRT,UTown,Monday,09:30:00,10 - 15 minutes,Satisfied,Satisfied,Satisfied,Satisfied,Satisfied,4,A2,IT,Opp KR MRT,Thursday,20:42:17,15 - 20 minutes,Satisfied,Satisfied,Satisfied,Satisfied,Satisfied,5.0,D2,UTown,Opp KR MRT,Wednesday,11:30:00,10 - 15 minutes,Satisfied,Satisfied,Satisfied,Satisfied,Satisfied,5.0,2nd,1st,3rd,5th,4th,2nd,6th,4th,3rd,5th,1st,"Travel time too long, Frequency of bus service...",Frequently,3rd,1st,4th,5th,6th,2nd,No,4,"No, service is consistent",NaN,Bigger bus size
2,10/2/2024 9:18:23,Undergraduate student,3 - 4 days a week,Commute to classes,"Monday, Wednesday, Thursday","1700 - 1800, 1800 - 1900, 2100 - 2200, 2200 - ...",A2,KR Bus Terminal,S17,Thursday,17:50:00,< 5 minutes,Very Satisfied,Satisfied,Satisfied,Satisfied,Very Satisfied,4,K,S17,Opp KR MRT,Wednesday,18:45:45,< 5 minutes,Satisfied,Satisfied,Satisfied,Satisfied,Satisfied,3.0,A1,KR MRT,Opp UHC,Wednesday,18:55:00,< 5 minutes,Very Satisfied,Very Satisfied,Very Satisfied,Satisfied,Satisfied,4.0,2nd,1st,3rd,4th,5th,1st,6th,5th,4th,3rd,2nd,"Capacity of the bus (Overcrowding), Proximity ...",Occasionally,1st,5th,6th,4th,3rd,2nd,No,3,"Yes, service improves/worsens (please specify ...",more in exams,na
3,10/2/2024 13:27:16,Undergraduate student,3 - 4 days a week,Commute to classes,"Monday, Thursday, Friday, Saturday, Sunday","1100 - 1200, 1200 - 1300, 1800 - 1900, 2100 - ...",A1,PGP Terminal,CLB,Friday,11:30:00,10 - 15 minutes,Very Satisfied,Very Satisfied,Very Satisfied,Neutral,Very Satisfied,4,D2,PGP Foyer,COM3,Sunday,12:45:45,15 - 20 minutes,Neutral,Very Satisfied,Very Satisfied,Neutral,Satisfied,3.0,A2,PGP Foyer,Ventus,Monday,11:30:00,15 - 20 minutes,Very dissatisfied,Very Satisfied,Very Satisfied,Neutral,Neutral,5.0,1st,3rd,2nd,5th,4th,1st,

### Preparing dataset

In [114]:
data_1 = pd.concat([data.loc[:, 'timestamp':'crowdedness_trip_1'], data.loc[:, 'usage_influence_convenience':]], axis=1)
data_2 = pd.concat([data.loc[:, 'timestamp':'travel_hours'], data.loc[:, 'ISB_Service_trip_2':'crowdedness_trip_2'], data.loc[:, 'usage_influence_convenience':]], axis=1)
data_3 = pd.concat([data.loc[:, 'timestamp':'travel_hours'], data.loc[:, 'ISB_Service_trip_3':]], axis=1)

data_1.rename(columns={'ISB_Service_trip_1': 'ISB_Service',
                            'bus_stop_board_trip_1': 'bus_stop_board',
                            'bus_stop_alight_trip_1': 'bus_stop_alight',
                            'day_of_the_week_trip_1': 'day_of_the_week',
                            'time_start_trip_1': 'time_start',
                            'travel_duration_trip_1': 'travel_duration',
                            'frequency_trip_1': 'frequency',
                            'punctuality_trip_1': 'punctuality',
                            'cleanliness_trip_1': 'cleanliness',
                            'safety_trip_1': 'safety',
                            'coverage_trip_1': 'coverage',
                            'crowdedness_trip_1': 'crowdedness'}, inplace=True)

data_2.rename(columns={'ISB_Service_trip_2': 'ISB_Service',
                            f'bus_stop_board_trip_2': 'bus_stop_board',
                            f'bus_stop_alight_trip_2': 'bus_stop_alight',
                            f'day_of_the_week_trip_2': 'day_of_the_week',
                            f'time_start_trip_2': 'time_start',
                            f'travel_duration_trip_2': 'travel_duration',
                            f'frequency_trip_2': 'frequency',
                            f'punctuality_trip_2': 'punctuality',
                            f'cleanliness_trip_2': 'cleanliness',
                            f'safety_trip_2': 'safety',
                            f'coverage_trip_2': 'coverage',
                            f'crowdedness_trip_2': 'crowdedness'}, inplace=True)

data_3.rename(columns={'ISB_Service_trip_3': 'ISB_Service',
                            f'bus_stop_board_trip_3': 'bus_stop_board',
                            f'bus_stop_alight_trip_3': 'bus_stop_alight',
                            f'day_of_the_week_trip_3': 'day_of_the_week',
                            f'time_start_trip_3': 'time_start',
                            f'travel_duration_trip_3': 'travel_duration',
                            f'frequency_trip_3': 'frequency',
                            f'punctuality_trip_3': 'punctuality',
                            f'cleanliness_trip_3': 'cleanliness',
                            f'safety_trip_3': 'safety',
                            f'coverage_trip_3': 'coverage',
                            f'crowdedness_trip_3': 'crowdedness'}, inplace=True)

data_combined = pd.concat([data_1, data_2, data_3], axis=0)
data_combined = data_combined[data_combined['ISB_Service'] != 'No trip']
data_combined['day_of_the_week'] = data_combined['day_of_the_week'].str.split(', ')
data_combined = data_combined.explode('day_of_the_week').reset_index(drop=True)

# Remove columns that need not be generated
data_combined.drop(columns=['timestamp', 'top_3_frustrations', 'seasonal_changes_specific', 'further_comments'], inplace=True)
data_combined

,role,frequency_of_travel,primary_purpose,travel_days,travel_hours,ISB_Service,bus_stop_board,bus_stop_alight,day_of_the_week,time_start,travel_duration,frequency,punctuality,cleanliness,safety,coverage,crowdedness,usage_influence_convenience,usage_influence_cost,usage_influence_lack_of_options,usage_influence_availability_of_parking,usage_influence_environmental,prioritize_frequency,prioritize_punctuality,prioritize_cleanliness,prioritize_safety,prioritize_bus_route_coverage,prioritize_crowdedness,not_able_to_get_on,additional_features_frequency,additional_features_seats,additional_features_cleanliness,additional_features_comfortable,additional_features_route_coverage,additional_features_updates,issues_with_quality_of_info,special_events,seasonal_changes
0,Undergraduate student,Daily,Commute to classes,"Monday, Tuesday, Wednesday, Thursday, Friday","0900 - 1000, 1000 - 1100, 1100 - 1200, 1200 - ...",A2,Opp NUSS,Ventus,Monday,09:30:00,15 - 20 minutes,Neutral,Neutral,Very Satisfied,Satisfied,Satisfied,3,1st,2nd,3rd,4th,5th,3rd,2nd,6th,5th,4th,1st,Occasionally,1st,4th,6th,5th,2nd,3rd,Yes,3,"No, service is consistent"
1,Undergraduate student,3 - 4 days a week,Commute to classes,"Monday, Wednesday, Thursday","0900 - 1000, 1100 - 1200, 1300 - 1400, 1500 - ...",D2,KR MRT,UTown,Monday,09:30:00,10 - 15 minutes,Satisfied,Satisfied,Satisfied,Satisfied,Satisfied,4,2nd,1st,3rd,5th,4th,2nd,6th,4th,3rd,5th,1st,Frequently,3rd,1st,4th,5th,6th,2nd,No,4,"No, service is consistent"
2,Undergraduate student,3 - 4 days a week,Commute to classes,"Monday, Wednesday, Thursday","1700 - 1800, 1800 - 1900, 2100 - 2200, 2200 - ...",A2,KR Bus Terminal,S17,Thursday,17:50:00,< 5 minutes,Very Satisfied,Satisfied,Satisfied,Satisfied,Very Satisfied,4,2nd,1st,3rd,4th,5th,1st,6th,5th,4th,3rd,2nd,Occasionally,1st,5th,6th,4th,3rd,2nd,No,3,"Yes, service improves/worsens (please specify ..."
3,Undergraduate student,3 - 4 days a week,Commute to classes,"Monday, Thursday, Friday, Saturday, Sunday","1100 - 1200, 1200 - 1300, 1800 - 1900, 2100 - ...",A1,PGP Terminal,CLB,Friday,11:30:00,10 - 15 minutes,Very Satisfied,Very Satisfied,Very Satisfied,Neutral,Very Satisfied,4,1st,3rd,2nd,5th,4th,1st,4th,5th,6th,3rd,2nd,Occasionally,1st,2nd,6th,5th,3rd,4th,No,3,"No, service is consistent"
4,Undergraduate student,1 - 2 days a week,Commute to classes,"Tuesday, Thursday, Friday","0700 - 0800, 1000 - 1100, 1100 - 1200, 1200 - ...",A1,KR MRT,LT27,Tuesday,07:50:00,< 5 minutes,Neutral,Satisfied,Satisfied,Satisfied,Satisfied,5,2nd,3rd,1st,5th,4th,2nd,3rd,6th,5th,1st,4th,Frequently,1st,2nd,6th,5th,3rd,4th,No,1,"Yes, service improves/worsens (please specify ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,Visitor,Less than once a week,Travelling to food establishments,"Monday, Tuesday, Wednesday, Thursday, Friday","1100 - 1200, 1200 - 1300",K,S17,Opp KR MRT,Wednesday,12:45:00,< 5 minutes,Neutral,Satisfied,Very Satisfied,Very Satisfied,Very Satisfied,3.0,2nd,1st,4th,5th,3rd,2nd,3rd,5th,4th,6th,1st,Frequently,1st,2nd,4th,5th,6th,3rd,No,3,"Yes, service improves/worsens (please specify ..."
236,Visitor,Less than once a week,Travelling to food establishments,"Monday, Tuesday, Wednesday, Thursday, Friday","1100 - 1200, 1200 - 1300",K,S17,Opp KR MRT,Thursday,12:45:00,< 5 minutes,Neutral,Satisfied,Very Satisfied,Very Satisfied,Very Satisfied,3.0,2nd,1st,4th,5th,3rd,2nd,3rd,5th,4th,6th,1st,Frequently,1st,2nd,4th,5th,6th,3rd,No,3,"Yes, service improves/worsens (please specify ..."
237,Visitor,Less than once a week,Travelling to food establishments,"Monday, Tuesday, Wednesday, Thursday, Friday","1100 - 1200, 1200 - 1300",K,S17,Opp KR MRT,Friday,12:45:00,< 5 minutes,Neutral,Satisfied,Very Satisfied,Very Satisfied,Very Satisfied,3.0,2nd,1st,4th,5th,3rd,2nd,3rd,5th,4th,6th,1st,Frequently,1st,2nd,4th,5th,6th,3rd,No,3,"Yes, service improves/worsens (please specify ..."
238,Undergraduate st

## Preparing data for generation

In [115]:
metadata = Metadata.detect_from_dataframe(
    data = data_combined,
    table_name='transport')

metadata.update_columns_metadata(
    column_metadata = {
        'travel_days' : {'sdtype': 'categorical'},
        'travel_hours' : {'sdtype': 'categorical'},
        'time_start' : {'sdtype': 'datetime', 'datetime_format': '%H:%M:%S'}
        }
)

metadata.validate()

metadata

{
    "tables": {
        "transport": {
            "columns": {
                "role": {
                    "sdtype": "categorical"
                },
                "frequency_of_travel": {
                    "sdtype": "categorical"
                },
                "primary_purpose": {
                    "sdtype": "categorical"
                },
                "travel_days": {
                    "sdtype": "categorical"
                },
                "travel_hours": {
                    "sdtype": "categorical"
                },
                "ISB_Service": {
                    "sdtype": "categorical"
                },
                "bus_stop_board": {
                    "sdtype": "categorical"
                },
                "bus_stop_alight": {
                    "sdtype": "categorical"
                },
                "day_of_the_week": {
                    "sdtype": "categorical"
                },
                "time_start": {
                    "

In [116]:
synthesizer = GaussianCopulaSynthesizer(metadata)
synthesizer.load_custom_constraint_classes(filepath='custom_constraints.py', class_names=['BusStopsCheck', 'TimeCheck'])

route_constraint = {
    'constraint_class': 'BusStopsCheck',
    'constraint_parameters': {
        'column_names':['ISB_Service', 'bus_stop_board', 'bus_stop_alight']
    }
}
time_constraint = {
    'constraint_class': 'TimeCheck',
    'constraint_parameters': {
        'column_names':['time_start']
    }
}

synthesizer.add_constraints(constraints=[route_constraint, time_constraint])

synthesizer.fit(data_combined)

synthetic_data = synthesizer.sample(num_rows=60000)

synthetic_data

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sdv/single_table/base.py:105: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(
Sampling rows: 100%|██████████| 60000/60000 [00:28<00:00, 2086.83it/s]


,role,frequency_of_travel,primary_purpose,travel_days,travel_hours,ISB_Service,bus_stop_board,bus_stop_alight,day_of_the_week,time_start,travel_duration,frequency,punctuality,cleanliness,safety,coverage,crowdedness,usage_influence_convenience,usage_influence_cost,usage_influence_lack_of_options,usage_influence_availability_of_parking,usage_influence_environmental,prioritize_frequency,prioritize_punctuality,prioritize_cleanliness,prioritize_safety,prioritize_bus_route_coverage,prioritize_crowdedness,not_able_to_get_on,additional_features_frequency,additional_features_seats,additional_features_cleanliness,additional_features_comfortable,additional_features_route_coverage,additional_features_updates,issues_with_quality_of_info,special_events,seasonal_changes
0,Undergraduate student,Daily,Commute to classes,"Tuesday, Thursday, Friday","1600 - 1700, 1700 - 1800",D1,IT,UTown,Thursday,11:25:53,15 - 20 minutes,Very dissatisfied,Neutral,Dissatisfied,Neutral,Very dissatisfied,2.0,1st,1st,3rd,5th,4th,1st,3rd,5th,1st,1st,1st,Frequently,1st,4th,6th,5th,3rd,3rd,Yes,2,"Yes, service improves/worsens (please specify ..."
1,Undergraduate student,Daily,Commute to classes,Thursday,"1100 - 1200, 1200 - 1300, 1800 - 1900, 2100 - ...",A2,IT,Opp NUSS,Tuesday,13:37:23,10 - 15 minutes,Neutral,Satisfied,Neutral,Satisfied,Satisfied,4.0,2nd,3rd,3rd,4th,5th,1st,3rd,6th,5th,2nd,6th,Frequently,1st,3rd,5th,6th,3rd,2nd,Yes,3,"Yes, service improves/worsens (please specify ..."
2,Visitor,3 - 4 days a week,Commute to classes,"Monday, Thursday, Friday","1100 - 1200, 1200 - 1300",K,KR MRT,CLB,Friday,11:31:55,< 5 minutes,Satisfied,Dissatisfied,Very Satisfied,Very Satisfied,Satisfied,2,2nd,1st,3rd,5th,5th,3rd,3rd,5th,4th,6th,2nd,Occasionally,1st,3rd,5th,6th,6th,3rd,Yes,2,"No, service is consistent"
3,Undergraduate student,1 - 2 days a week,Commute to classes,"Monday, Tuesday, Thursday",0900 - 1000,D2,COM3,LT27,Sunday,15:22:10,< 5 minutes,Dissatisfied,Neutral,Very Satisfied,Satisfied,Satisfied,4,1st,3rd,3rd,4th,4th,3rd,5th,6th,4th,6th,4th,Frequently,1st,2nd,5th,5th,4th,2nd,Yes,1,"Yes, service improves/worsens (please specify ..."
4,Undergraduate student,3 - 4 days a week,Commute to classes,Friday,"1000 - 1100, 1100 - 1200, 1200 - 1300, 1300 - ...",A1,BIZ2,CLB,Wednesday,14:48:32,10 - 15 minutes,Dissatisfied,Very Satisfied,Satisfied,Satisfied,Neutral,5,4th,5th,3rd,1st,3rd,2nd,1st,6th,4th,2nd,1st,Occasionally,2nd,4th,6th,6th,3rd,2nd,Yes,2,"No, service is consistent"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,Undergraduate student,Less than once a week,Commute to classes,"Monday, Tuesday, Wednesday, Thursday","1200 - 1300, 1300 - 1400, 1400 - 1500",A1,KR Bus Terminal,LT27,Wednesday,11:08:08,5 - 10 minutes,Dissatisfied,Satisfied,Satisfied,Neutral,Neutral,5.0,3rd,1st,1st,5th,4th,1st,3rd,6th,5th,3rd,2nd,Frequently,1st,4th,4th,5th,3rd,5th,No,3,"Yes, service improves/worsens (please specify ..."
59996,Undergraduate student,3 - 4 days a week,"Accessing campus facilities (gym, library etc.)","Monday, Tuesday, Wednesday, Thursday, Friday","0800 - 0900, 1100 - 1200, 1300 - 1400, 1500 - ...",A1,KR MRT,CLB,Thursday,13:34:00,10 - 15 minutes,Satisfied,Dissatisfied,Very Satisfied,Very dissatisfied,Satisfied,2,2nd,3rd,1st,3rd,2nd,2nd,2nd,4th,4th,1st,4th,Occasionally,2nd,4th,4th,6th,6th,6th,Yes,4,"No, service is consistent"
59997,Undergraduate student,1 - 2 days a week,Travelling to extracurricular activities,"Monday, Tuesday, Wednesday, Thursday",0900 - 1000,D2,PGP Terminal,UTown,Tuesday,09:38:07,15 - 20 minutes,Neutral,Satisfied,Very Satisfied,Neutral,Satisfied,2,1st,2nd,2nd,4th,4th,1st,2nd,6th,5th,1st,2nd,Occasionally,2nd,1st,6th,5th,6th,3rd,No,3,"No, service is consistent"
59998,Undergraduate student,3 - 4 days a week,Commute to classes,"Monday, Tuesday, Wednesday, Thursday, Friday","0900 - 1000, 1100 - 1200, 1300 - 1400, 1500 - ...",E,UTown,UTown,Sunday,14:33:16,5 -

### Cleaning synthetic data

In [120]:
created_data = synthetic_data.copy()
all_data = pd.concat([data_combined, created_data], axis=0)

# Map ordinal features
rank_mapping = {'1st': 1,
                '2nd': 2,
                '3rd': 3,
                '4th': 4,
                '5th': 5,
                '6th': 6}

satisfaction_mapping = {
    'Very dissatisfied': 1,
    'Dissatisfied': 2,
    'Neutral': 3,
    'Satisfied': 4,
    'Very Satisfied': 5
}

to_map_rank = ['usage_influence_convenience', 'usage_influence_cost', 'usage_influence_lack_of_options', 'usage_influence_availability_of_parking', 'usage_influence_environmental',
               'prioritize_frequency', 'prioritize_punctuality', 'prioritize_cleanliness', 'prioritize_safety', 'prioritize_bus_route_coverage', 'prioritize_crowdedness', 
               'additional_features_frequency', 'additional_features_seats', 'additional_features_cleanliness', 'additional_features_comfortable', 
               'additional_features_route_coverage', 'additional_features_updates']

to_map_satisfaction = ['frequency', 'punctuality', 'cleanliness', 'safety', 'coverage']

for c in to_map_rank:
    all_data[c] = synthetic_data[c].map(rank_mapping)

for c in to_map_satisfaction:
    all_data[c] = synthetic_data[c].map(satisfaction_mapping)

# Convert crowdedness column to integer
all_data['crowdedness'] = pd.to_numeric(synthetic_data['crowdedness']).astype(int)

all_data = all_data.reset_index(drop=True)
all_data

,role,frequency_of_travel,primary_purpose,travel_days,travel_hours,ISB_Service,bus_stop_board,bus_stop_alight,day_of_the_week,time_start,travel_duration,frequency,punctuality,cleanliness,safety,coverage,crowdedness,usage_influence_convenience,usage_influence_cost,usage_influence_lack_of_options,usage_influence_availability_of_parking,usage_influence_environmental,prioritize_frequency,prioritize_punctuality,prioritize_cleanliness,prioritize_safety,prioritize_bus_route_coverage,prioritize_crowdedness,not_able_to_get_on,additional_features_frequency,additional_features_seats,additional_features_cleanliness,additional_features_comfortable,additional_features_route_coverage,additional_features_updates,issues_with_quality_of_info,special_events,seasonal_changes
0,Undergraduate student,Daily,Commute to classes,"Monday, Tuesday, Wednesday, Thursday, Friday","0900 - 1000, 1000 - 1100, 1100 - 1200, 1200 - ...",A2,Opp NUSS,Ventus,Monday,09:30:00,15 - 20 minutes,1,3,2,3,1,2,1,1,3,5,4,1,3,5,1,1,1,Occasionally,1,4,6,5,3,3,Yes,3,"No, service is consistent"
1,Undergraduate student,3 - 4 days a week,Commute to classes,"Monday, Wednesday, Thursday","0900 - 1000, 1100 - 1200, 1300 - 1400, 1500 - ...",D2,KR MRT,UTown,Monday,09:30:00,10 - 15 minutes,3,4,3,4,4,4,2,3,3,4,5,1,3,6,5,2,6,Frequently,1,3,5,6,3,2,No,4,"No, service is consistent"
2,Undergraduate student,3 - 4 days a week,Commute to classes,"Monday, Wednesday, Thursday","1700 - 1800, 1800 - 1900, 2100 - 2200, 2200 - ...",A2,KR Bus Terminal,S17,Thursday,17:50:00,< 5 minutes,4,2,5,5,4,2,2,1,3,5,5,3,3,5,4,6,2,Occasionally,1,3,5,6,6,3,No,3,"Yes, service improves/worsens (please specify ..."
3,Undergraduate student,3 - 4 days a week,Commute to classes,"Monday, Thursday, Friday, Saturday, Sunday","1100 - 1200, 1200 - 1300, 1800 - 1900, 2100 - ...",A1,PGP Terminal,CLB,Friday,11:30:00,10 - 15 minutes,2,3,5,4,4,4,1,3,3,4,4,3,5,6,4,6,4,Occasionally,1,2,5,5,4,2,No,3,"No, service is consistent"
4,Undergraduate student,1 - 2 days a week,Commute to classes,"Tuesday, Thursday, Friday","0700 - 0800, 1000 - 1100, 1100 - 1200, 1200 - ...",A1,KR MRT,LT27,Tuesday,07:50:00,< 5 minutes,2,5,4,4,3,5,4,5,3,1,3,2,1,6,4,2,1,Frequently,2,4,6,6,3,2,No,1,"Yes, service improves/worsens (please specify ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60235,Undergraduate student,Less than once a week,Commute to classes,"Monday, Tuesday, Wednesday, Thursday","1200 - 1300, 1300 - 1400, 1400 - 1500",A1,KR Bus Terminal,LT27,Wednesday,11:08:08,5 - 10 minutes,2,4,4,3,3,5,3,1,1,5,4,1,3,6,5,3,2,Frequently,1,4,4,5,3,5,No,3,"Yes, service improves/worsens (please specify ..."
60236,Undergraduate student,3 - 4 days a week,"Accessing campus facilities (gym, library etc.)","Monday, Tuesday, Wednesday, Thursday, Friday","0800 - 0900, 1100 - 1200, 1300 - 1400, 1500 - ...",A1,KR MRT,CLB,Thursday,13:34:00,10 - 15 minutes,4,2,5,1,4,2,2,3,1,3,2,2,2,4,4,1,4,Occasionally,2,4,4,6,6,6,Yes,4,"No, service is consistent"
60237,Undergraduate student,1 - 2 days a week,Travelling to extracurricular activities,"Monday, Tuesday, Wednesday, Thursday",0900 - 1000,D2,PGP Terminal,UTown,Tuesday,09:38:07,15 - 20 minutes,3,4,5,3,4,2,1,2,2,4,4,1,2,6,5,1,2,Occasionally,2,1,6,5,6,3,No,3,"No, service is consistent"
60238,Undergraduate student,3 - 4 days a week,Commute to classes,"Monday, Tuesday, Wednesday, Thursday, Friday","0900 - 1000, 1100 - 1200, 1300 - 1400, 1500 - ...",E,UTown,UTown,Sunday,14:33:16,5 - 10 minutes,3,3,4,3,4,5,2,2,1,4,5,3,3,5,5,6,1,Occasionally,1,3,6,5,3,2,Yes,4,"No, service is consistent"


### Writing synthetic data to CSV

In [118]:
# all_data.to_csv('synthetic_data.csv', index=False)